In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
#Load your dataset
df = pd.read_csv("../../0_DataPreparation/initialdata/merged_data_temperature+holidays+weather_impressions.csv") # 

#Ensure the 'Datum' column is in datetime format
df['Datum'] = pd.to_datetime(df['Datum'])

#Define time ranges
train_start = '2013-07-01'
train_end = '2017-07-31'
val_start = '2017-08-01'
val_end = '2018-07-31'
test_start = '2018-08-01'
test_end = '2019-07-30'

#Split data
train_data = df[(df['Datum'] >= train_start) & (df['Datum'] <= train_end)]
validation_data = df[(df['Datum'] >= val_start) & (df['Datum'] <= val_end)]
test_data = df[(df['Datum'] >= test_start) & (df['Datum'] <= test_end)]

#Optional: print shapes
print("Train shape:", train_data.shape)
print("Validation shape:", validation_data.shape)
print("Test shape:", test_data.shape)

Train shape: (7517, 15)
Validation shape: (1839, 15)
Test shape: (351, 15)


In [3]:
columns = train_data.columns.tolist()
columns

['Datum',
 'Weekday',
 'Bewoelkung',
 'Temperatur',
 'Windgeschwindigkeit',
 'Wettercode',
 'id',
 'Warengruppe',
 'Umsatz',
 'KielerWoche',
 'AverageTemp',
 'Temp_Deviation',
 'Temp_vs_Avg',
 'Weather_Impression',
 'Is_Holiday']

In [4]:
#remove all non-numeric columns for correlation
columns = [col for col in columns if train_data[col].dtype in ['float64', 'int64']]
train_data[columns].corr()

,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,id,Umsatz,KielerWoche,AverageTemp,Temp_Deviation,Is_Holiday
Bewoelkung,1.000000,-0.376459,0.042912,0.549314,0.041113,-0.085921,NaN,-0.270596,-0.283291,-0.050103
Temperatur,-0.376459,1.000000,0.003471,-0.204801,-0.095729,0.216570,NaN,0.892810,0.398119,0.006644
Windgeschwindigkeit,0.042912,0.003471,1.000000,0.151729,0.027233,0.011454,NaN,0.014588,-0.022016,0.025854
Wettercode,0.549314,-0.204801,0.151729,1.000000,0.041823,-0.046785,NaN,-0.134065,-0.180885,-0.031755
id,0.041113,-0.095729,0.027233,0.041823,1.000000,-0.097429,NaN,-0.068435,-0.072883,0.022955
Umsatz,-0.085921,0.216570,0.011454,-0.046785,-0.097429,1.000000,NaN,0.235087,0.001433,0.047747
KielerWoche,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AverageTemp,-0.270596,0.892810,0.014588,-0.134065,-0.068435,0.235087,NaN,1.000000,-0.057753,0.022092
Temp_Deviation,-0.283291,0.398119,-0.022016,-0.180885,-0.072883,0.001433,NaN,-0.057753,1.000000,-0.030266
Is_Holiday,-0.050103,0.006644,0.025854,-0.031755,0.022955,0.047747,NaN,0.022092,-0.030266,1.000000


Third Model including Warengruppe, Holidays, Weekday and Temperature

In [9]:
# Convert 'Datum' to datetime and extract weekday name
train_data['Datum'] = pd.to_datetime(train_data['Datum'])
train_data['Wochentag'] = train_data['Datum'].dt.day_name()

# Create dummy variables for categorical features
X = sm.add_constant(
    pd.get_dummies(train_data[["Warengruppe", "Is_Holiday", "Wochentag", "Temperatur"]], 
                   drop_first=True, dtype=int)
)
Y = train_data['Umsatz']

# Drop rows with NaN values in predictors or target
valid_idx = Y.notna() & X.notna().all(axis=1)
Y_clean = Y[valid_idx]
X_clean = X[valid_idx]

# Fit linear regression model
model = sm.OLS(Y_clean, X_clean)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Umsatz   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     1449.
Date:                Tue, 03 Jun 2025   Prob (F-statistic):               0.00
Time:                        09:01:28   Log-Likelihood:                -43313.
No. Observations:                7487   AIC:                         8.665e+04
Df Residuals:                    7473   BIC:                         8.675e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     60

/tmp/ipykernel_19807/2069508393.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Datum'] = pd.to_datetime(train_data['Datum'])
/tmp/ipykernel_19807/2069508393.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Wochentag'] = train_data['Datum'].dt.day_name()


In [16]:
# Load test data
test_data = pd.read_csv("../Linear_regression_Lukas/test.csv")
test_data['Datum'] = pd.to_datetime(test_data['Datum'])

# Map numeric 'Warengruppe' codes to product names
warengruppe_mapping = {
    1: 'Brot',
    2: 'Brötchen',
    3: 'Croissant',
    4: 'Konditorei',
    5: 'Kuchen',
    6: 'Saisonbrot'
}
test_data['Warengruppe'] = test_data['Warengruppe'].map(warengruppe_mapping)

# Define holiday dates and create 'Is_Holiday' flag
holidays = pd.to_datetime([
    # 2012–2019 holidays as in training
])
test_data['Is_Holiday'] = test_data['Datum'].isin(holidays).astype(int)

# Add weekday name
test_data['Wochentag'] = test_data['Datum'].dt.day_name()

# Add temperature info from df
test_data = test_data.merge(df[['Datum', 'Temperatur']], on='Datum', how='left')

# find days where temperature is missing
missing_temps = test_data[test_data['Temperatur'].isna()]

# add values for missing temperatures
test_data['Temperatur'].fillna(method='ffill', inplace=True)



/tmp/ipykernel_19807/2371747917.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data['Temperatur'].fillna(method='ffill', inplace=True)
/tmp/ipykernel_19807/2371747917.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_data['Temperatur'].fillna(method='ffill', inplace=True)


In [17]:
# Create dummy variables for test data
X_test = pd.get_dummies(test_data[["Warengruppe", "Is_Holiday", "Wochentag", "Temperatur"]], drop_first=True, dtype=int)
X_test = sm.add_constant(X_test)

# Add missing columns (from training set) to test data, filled with 0
for col in X_clean.columns:
    if col not in X_test.columns:
        X_test[col] = 0

# Reorder columns to match training data
X_test = X_test[X_clean.columns]

# Predict 'Umsatz' using the trained model
preds = results.predict(X_test)

In [18]:
predicted = results.predict(X_clean)
actual = Y_clean
residuals = actual - predicted

comparison_df = pd.DataFrame({
    'Predicted': predicted,
    'Actual': actual,
    'Residuals': residuals
})
print(comparison_df.head())

      Predicted  Actual   Residuals
394  128.451135  148.83   20.378865
395  417.615146  535.86  118.244854
396  171.207446  201.20   29.992554
397   95.334385   65.89  -29.444385
398  287.974557  317.48   29.505443


In [19]:
# Create submission file
submission = pd.DataFrame({
    'id': test_data['id'],
    'Umsatz': preds
})
submission.to_csv("sub3.csv", index=False)